In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
# Define a user-agent header to identify your scraper
user_agent = "MyWebScraper/1.0 (+https://github.com/Adesuaayo/goodreads_webscraper)"

In [3]:
book_titles = []
authors = []
avg_ratings = []
ratings = []
published_years = []
editions = []

In [4]:
pages_to_scrape = 1000

# Specify the delay between requests in seconds (e.g., 2 seconds)
request_delay = 3

# Loop through the pages to scrape
for page in range(1, pages_to_scrape + 1):
    
    # Construct the URL for the current page
    url = "https://www.goodreads.com/search?page=" + str(page) + "&q=data+analysis&qid=LtFU5Ojc1o&tab=books"
   
    try:
        # Send an HTTP GET request to the URL with the user-agent header
        headers = {"User-Agent": user_agent}
        response = requests.get(url, headers=headers).text

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response, "html5lib")
    
        # Check for server errors or maintenance
        if soup.title and "service unavailable" in soup.title.text.lower():
            print(f"Server error on page {page}. Skipping...")
            continue

        # Select the table containing the list of books
        table = soup.find_all("tbody")[0]

        # Loop through the rows of the table
        for row in table.find_all("tr"):
            cells = row.find_all("td")[1]

            # Extract book title
            title = cells.find("a").find("span").text
            book_titles.append(title)

            # Extract author's name
            author = cells.find("a", class_="authorName").text
            authors.append(author)
            

            #rating
            all_ratings = cells.find_all('span', class_ = 'minirating')
            all_ratings_text = all_ratings[0].text.strip()
            pattern_2 = re.compile(r"(\d\.?\d*)\savg")
            avg_ratings.append(pattern_2.search(all_ratings_text).group(1))

            #n_ratings
            pattern_4 = re.compile(r"(\d\,?\d*) rating")
            ratings_matches = pattern_4.search(all_ratings_text)
            ratings.append(ratings_matches.group(1) if ratings_matches else 0)  


#             # Extract average rating
#             avg_rating = cells.find("span", class_="greyText smallText uitext").text.split()[0]
#             avg_ratings.append(avg_rating)

#             # Extract rating
#             rating = cells.find("span", class_="greyText smallText uitext").text.split()[4]
#             ratings.append(rating)

            # Extract published year, handling cases where it may not be in the expected format
            year_info = cells.find("span", class_="greyText smallText uitext").text.split()
            year = None
            for item in year_info:
                if item.isdigit() and len(item) == 4:
                    year = item
                    break
            if year:
                published_years.append(year)
            else:
                published_years.append(0)  # Handle cases where year is not found

            # Extract edition information
            edition = cells.find("span", class_="greyText smallText uitext").text.split()[-2]
            editions.append(edition)

        # Sleep to add a delay between requests
        time.sleep(request_delay)
    
    except requests.exceptions.RequestException as e:
        # Handle HTTP request errors (e.g., connection issues)
        print(f"Error on page {page}: {e}")

    except IndexError as e:
        # Handle "list index out of range" error
        print(f"Index error on page {page}: {e}")

    except Exception as e:
        # Handle other unexpected errors
        print(f"Unexpected error on page {page}: {e}")

Error on page 7: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))
Index error on page 29: list index out of range
Index error on page 30: list index out of range
Error on page 33: HTTPSConnectionPool(host='www.goodreads.com', port=443): Max retries exceeded with url: /search?page=33&q=data+analysis&qid=LtFU5Ojc1o&tab=books (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
Error on page 50: HTTPSConnectionPool(host='www.goodreads.com', port=443): Max retries exceeded with url: /search?page=50&q=data+analysis&qid=LtFU5Ojc1o&tab=books (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
Index error on page 99: list index out of range
Error on page 102: HTTPSConnectionPool(host='www.goodreads.com', port=443): Max

In [5]:
# After scraping all pages, we create a DataFrame from the collected data
data = {
    "Title": book_titles,
    "Author": authors,
    "Average Rating": avg_ratings,
    "Rating": ratings,
    "Year Published": published_years,
    "Edition": editions
}

In [6]:
goodreads = pd.DataFrame(data)

# Display the first five rows of the dataframe
goodreads.head()

,Title,Author,Average Rating,Rating,Year Published,Edition
0,Python for Data Analysis,Wes McKinney,4.16,"2,178",2011,38
1,Bayesian Data Analysis,Andrew Gelman,4.22,522,1995,18
2,Data Analysis with Open Source Tools: A Hands-...,Philipp K. Janert,4.08,313,2010,10
3,"R Cookbook: Proven Recipes for Data Analysis, ...",Paul Teetor,4.05,439,2011,7
4,Data Analysis Using Regression and Multilevel/...,Andrew Gelman,4.35,274,2006,12


In [7]:
len(goodreads)

15678

In [8]:
goodreads.to_csv("data_analysis_books.csv", index=False)